In [5]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from keras import backend as K

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11663062827186235081
xla_global_id: -1
]


In [17]:
def conv_block(x, num_filters):
    x = Conv2D(num_filters, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [48]:
def build_model(shape):
    num_filters = [64, 128, 256, 512]
    inputs = Input(shape)

    skip_x = []
    x = inputs
    
    # ENCODER
    for f in num_filters:
        x = conv_block(x, f)
        skip_x.append(x)
        x = MaxPooling2D((2, 2))(x)

    # BRIDGE
    x = conv_block(x, 1024)

    num_filters.reverse()
    skip_x.reverse()

    # DECODER
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)
        xs = skip_x[i]

        # Apply cropping or padding to match shapes
        diff_h = xs.shape[1] - x.shape[1]
        diff_w = xs.shape[2] - x.shape[2]

        if diff_h > 0:
            x = ZeroPadding2D(((0, diff_h), (0, 0)))(x)
        elif diff_h < 0:
            xs = ZeroPadding2D(((0, -diff_h), (0, 0)))(xs)

        if diff_w > 0:
            x = ZeroPadding2D(((0, 0), (0, diff_w)))(x)
        elif diff_w < 0:
            xs = ZeroPadding2D(((0, 0), (0, -diff_w)))(xs)

        x = Concatenate()([x, xs])
        x = conv_block(x, f)
    
    # OUTPUT
    x = Conv2D(1, (1, 1), padding="same")(x)  # Output with 1 channel
    x = Activation("sigmoid")(x)

    return Model(inputs, x)

In [49]:
input_shape = (300, 130, 3)  
model = build_model(input_shape)
print(model.summary())

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 300, 130,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_166 (Conv2D) │ (None, 300, 130,  │      1,792 │ input_layer_14[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 300, 130,  │        256 │ conv2d_166[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_155      │ (None, 300, 130,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_167 (Conv2D) │ (None, 300, 130,  │     36,928 │ activation_155[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 300, 130,  │        256 │ conv2d_167[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_156      │ (None, 300, 130,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_48    │ (None, 150, 65,   │          0 │ activation_156[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_168 (Conv2D) │ (None, 150, 65,   │     73,856 │ max_pooling2d_48… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 150, 65,   │        512 │ conv2d_168[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_157      │ (None, 150, 65,   │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_169 (Conv2D) │ (None, 150, 65,   │    147,584 │ activation_157[0… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 150, 65,   │        512 │ conv2d_169[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_158      │ (None, 150, 65,   │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_49    │ (None, 75, 32,    │          0 │ activation_158[0… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_170 (Conv2D) │ (None, 75, 32,    │    295,168 │ max_pooling2d_49… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 75, 32,    │      1,024 │ conv2d_170[0][0]

 Total params: 31,402,497 (119.79 MB)

 Trainable params: 31,390,721 (119.75 MB)

 Non-trainable params: 11,776 (46.00 KB)

None


In [6]:
import numpy as np

trainimageNP = np.load("../dataset/augmented/feed/train_img.npy")
trainmaskNP = np.load("../dataset/augmented/feed/train_mask.npy")
validimageNP = np.load("../dataset/augmented/feed/valid_img.npy")
validmaskNP = np.load("../dataset/augmented/feed/valid_mask.npy")

In [51]:
print(trainimageNP.shape)
print(trainmaskNP.shape)
print(validimageNP.shape)
print(validmaskNP.shape)

(2091, 300, 130, 3)
(2091, 300, 130)
(233, 300, 130, 3)
(233, 300, 130)


In [67]:
lr = 1e-4
batchSize = 32
epochs = 50

In [68]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# Compile the model
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Calculate steps per epoch and validation steps
stepsPerEpoch = len(trainimageNP) // batchSize
validationSteps = len(validimageNP) // batchSize

# Define the best model file path
best_model_file = "../dataset/augmented/unet-cnn.keras"

# Define callbacks
callbacks = [
    ModelCheckpoint(best_model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.1, verbose=1, min_lr=1e-7),
    EarlyStopping(monitor="val_loss", patience=20, verbose=1, mode="min")
]

# Train the model
history = model.fit(trainimageNP, trainmaskNP,
                    batch_size=batchSize,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(validimageNP, validmaskNP),
                    validation_steps=validationSteps,
                    steps_per_epoch=stepsPerEpoch,
                    shuffle=True,
                    callbacks=callbacks
                    )

Epoch 1/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 48s/step - accuracy: 0.9732 - loss: 0.0718 
Epoch 1: val_loss improved from inf to 0.10463, saving model to ../dataset/augmented/unet-cnn.keras
65/65 ━━━━━━━━━━━━━━━━━━━━ 3272s 49s/step - accuracy: 0.9732 - loss: 0.0717 - val_accuracy: 0.9608 - val_loss: 0.1046 - learning_rate: 1.0000e-04
Epoch 2/50
 1/65 ━━━━━━━━━━━━━━━━━━━━ 10:58 10s/step - accuracy: 0.9772 - loss: 0.0597

c:\Users\kimbe\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)



Epoch 2: val_loss improved from 0.10463 to 0.10285, saving model to ../dataset/augmented/unet-cnn.keras
65/65 ━━━━━━━━━━━━━━━━━━━━ 14s 66ms/step - accuracy: 0.9772 - loss: 0.0597 - val_accuracy: 0.9598 - val_loss: 0.1029 - learning_rate: 1.0000e-04
Epoch 3/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 40s/step - accuracy: 0.9819 - loss: 0.0503 
Epoch 3: val_loss improved from 0.10285 to 0.06168, saving model to ../dataset/augmented/unet-cnn.keras
65/65 ━━━━━━━━━━━━━━━━━━━━ 2697s 40s/step - accuracy: 0.9819 - loss: 0.0502 - val_accuracy: 0.9779 - val_loss: 0.0617 - learning_rate: 1.0000e-04
Epoch 4/50
 1/65 ━━━━━━━━━━━━━━━━━━━━ 9:07 9s/step - accuracy: 0.9853 - loss: 0.0402
Epoch 4: val_loss did not improve from 0.06168
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.9853 - loss: 0.0402 - val_accuracy: 0.9776 - val_loss: 0.0644 - learning_rate: 1.0000e-04
Epoch 5/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 27s/step - accuracy: 0.9877 - loss: 0.0352 
Epoch 5: val_loss improved from 0.06168 to 0.05138, s

In [12]:
import cv2

model = tf.keras.models.load_model("../dataset/augmented/unet-cnn.keras")

width = 130
height = 300

imgpath = "../dataset/annotations2/0413594-1.tiff"
img = cv2.imread(imgpath)
img2 = cv2.resize(img, (width, height))
img2 = img2/255.0
img3 = np.expand_dims(img2, axis=0)

pred = model.predict(img3)
resultMask = pred[0]

print(resultMask)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[[0.00663859]
  [0.00560077]
  [0.00384687]
  ...
  [0.00242746]
  [0.00227475]
  [0.00160943]]

 [[0.00376154]
  [0.00373219]
  [0.00344169]
  ...
  [0.00153265]
  [0.00231978]
  [0.00168021]]

 [[0.00289578]
  [0.00197902]
  [0.0028565 ]
  ...
  [0.00178062]
  [0.00149872]
  [0.00191482]]

 ...

 [[0.0030442 ]
  [0.00123614]
  [0.00119245]
  ...
  [0.00214818]
  [0.0023466 ]
  [0.00249374]]

 [[0.00607744]
  [0.00295033]
  [0.00173992]
  ...
  [0.00197823]
  [0.00319493]
  [0.00408875]]

 [[0.00510855]
  [0.004769  ]
  [0.00233243]
  ...
  [0.00448683]
  [0.01021888]
  [0.00764655]]]


In [13]:
resultMask[resultMask <=0.5] = 0
resultMask[resultMask > 0.5] = 255

scale_percent = 20

w = int(img.shape[1] * scale_percent/100)
h = int(img.shape[0] * scale_percent/100)

dim = (w,h)

img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
mask = cv2.resize(resultMask, dim, interpolation = cv2.INTER_AREA)

cv2.imshow("image", img)
cv2.imshow("mask", mask)
cv2.waitKey(0)

-1